In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import json

In [37]:
gmail = ['']
password = ['']

In [38]:
def login(browser):
    browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
    time.sleep(5)
    browser.find_element("name", "session_key").send_keys('huynhvananh308@gmail.com' + Keys.RETURN)
    browser.find_element("name", "session_password").send_keys('Jackson308.' + Keys.RETURN)
    time.sleep(5)

In [39]:
# Import dataframe into MySQL
import sqlalchemy
def etl_mysql():
    database_username = 'root'
    database_password = ''
    database_ip       = 'localhost'
    database_name     = 'LINKEDIN_USER'
    database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    return database_connection

In [40]:
conn = etl_mysql()
import pandas as pd
query = "select * from users"
users = pd.read_sql(query, conn)

In [41]:
def user_scrape_profile(browser, proFileLink):
    browser.get(proFileLink)
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(2)
    proFileLink = proFileLink.split('?')[0] + '/'
    browser.get(proFileLink + 'details/experience/')
    time.sleep(5)
    soup_exp = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(2)
    browser.get(proFileLink + 'details/skills/')
    time.sleep(5)
    soup_skill = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(30)
    return soup, soup_exp, soup_skill

In [42]:
def experiences(soup_exp):
    info_ex = soup_exp.find_all('li', class_ = 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated')
    experiences = {}
    if(len(info_ex) > 0):
        for i in range(len(info_ex)):
            job_title = info_ex[i].find_all('span', class_ = 'mr1 t-bold')
            job_title = list(map(lambda x: replace_(x.find_all('span')[0].text), job_title))
            company = info_ex[i].find_all('span', class_ ='t-14 t-normal')
            duration = info_ex[i].find_all('span', class_ ='t-14 t-normal t-black--light')
            if(len(job_title) == len(company) and (len(duration) -1 ) == len(job_title)):
                company = list(map(lambda x: replace_(x.find_all('span')[0].text), company))
                duration_ = replace_(duration[0].find_all('span')[0].text.strip())
                location = replace_(duration[1].find_all('span')[0].text.strip())
                res = {'Job Title':job_title, 'Company': company, 'Duration': duration_,'Location': location  }
            elif(len(job_title) != len(company) and (len(duration) -1 ) == len(job_title)):
                duration_ = replace_(duration[0].find_all('span')[0].text.strip())
                location = replace_(duration[1].find_all('span')[0].text.strip())
                res = {'Job Title':job_title, 'Company': "Null", 'Duration': duration_,'Location': location  }
            else:
                company = list(map(lambda x: replace_(x.find_all('span')[0].text), company))
                res = {"Job Title":job_title, "Company": company, 'Duration': "Null","Location": "Null"  }
            experiences[i] = res
    return json.dumps(experiences)

In [43]:
def skills(soup_skill):
    info_skill = soup_skill.find_all('span', class_ = 'mr1 t-bold')
    skills_ = {}
    if(len(info_skill) > 0): 
        for i in range(len(info_skill)):
            skills_[i] = replace_(info_skill[i].find_all('span')[0].text)
    return json.dumps(skills_)

In [44]:
def honors(honor_info):
    honors_ = {}
    honors = honor_info.find_all('span', attrs={'class': 'mr1 t-bold'})
    honors_issues = honor_info.find_all('span', attrs={'class': 't-14 t-normal'})
    honors = list(map(lambda x: replace_(x.find_all('span')[0].text), honors) )
    honors_issues = list(map(lambda x: replace_(x.find_all('span',attrs={'class': 'visually-hidden'})[0].text), honors_issues))
    if(len(honors) == len(honors_issues) and len(honors) != 0):
        for j in range(len(honors)):
            res = {"HonorName": honors[j], "Issue": honors_issues[j] }
            honors_[j] = res
    return json.dumps(honors_)

In [45]:
def educations(education_info):
    educations_ = {}
    educations = education_info.find_all('span', attrs={'class': 'mr1 hoverable-link-text t-bold'})
    educations = list(map(lambda x: x.find_all('span')[0].text, educations))
    majors = education_info.find_all('span', attrs={'class': 't-14 t-normal'})
    majors = list(map(lambda x: x.find_all('span',attrs={'class': 'visually-hidden'})[0].text, majors))
    durations = education_info.find_all('span', attrs={'class': 't-14 t-normal t-black--light'})
    durations = list(map(lambda x: x.find_all('span',attrs={'class': 'visually-hidden'})[0].text, durations))
    for j in range(len(educations)):
        if(len(educations) == len(majors) == len(durations)):
            res = {"School": replace_(educations[j]), "Major": replace_(majors[j]), "Duration":replace_(durations[j])  }
        elif (len(educations) == len(majors) and len(educations)!= len(durations)):
            print(educations[j])
            res = {"School": replace_(educations[j]), "Major": replace_(majors[j]), "Duration": "Null"}
        elif (len(educations) == len(durations) and len(educations)!= len(majors)):
            res = {"School": replace_(educations[j]), "Major": "Null", "Duration":replace_(durations[j])  }
        else:
             res = {"School": replace_(educations[j]), "Major": "Null", "Duration": "Null" }
        educations_[j] = res
        
    return json.dumps(educations_)

In [46]:
def replace_(str_):
    return str_.replace("'", "").replace("\t", "").replace("\uf0a7\t", "").replace("'", "").replace('"', '').replace(u'\u00b7', "").replace(u'\u00c7', "").replace(u'\00c3', "").replace(u'\u00d5', "").replace('\n', "").replace(u'\u00fc', "")


In [47]:
def publications(publication_info):
    pubications_ = {}
    publications = publication_info.find_all('span', attrs={'class': 'mr1 t-bold'})
    publications = list(map(lambda x: replace_(x.find_all('span')[0].text), publications))
    pub_urls = publication_info.find_all('a')
    pub_urls = list(map(lambda x: x.get('href'), pub_urls)) 
    for i in range(len(publications)):
        if(len(publications) == len(pub_urls)):
            res = {'Name' : publications[i], "URL" : pub_urls[i] }
        else:
            res = {'Name' : publications[i], "URL" : 'Null' }
        pubications_[i] = res
    return json.dumps(pubications_)

In [48]:
def extract_profile_sections(soup,soup_exp,soup_skill):
    name_ = soup.find_all('h1', class_ = 'text-heading-xlarge inline t-24 v-align-middle break-words')
    cur_pos_ = soup.find_all('div', attrs={'class': 'text-body-medium break-words'})
    position_ = soup.find_all('span', attrs={'class': 'text-body-small inline t-black--light break-words'})
    about_ = soup.find_all('div', attrs={'class': 'display-flex ph5 pv3'})
    img_ = soup.find_all('img', class_ = 'pv-top-card-profile-picture__image pv-top-card-profile-picture__image--show ember-view')
    if(len(name_) != 0): name_ = replace_(name_[0].text)
    if(len(cur_pos_) != 0): cur_pos_ = replace_(cur_pos_[0].text.strip())
    if(len(position_) != 0):  position_ = replace_(position_[0].text.strip())
    if(len(about_) != 0) : about_ = replace_(about_[0].text.strip())
    if(len(img_) != 0) : img_ = img_[0].get('src')
    
    #extract all section
    info = soup.find_all('section', attrs={'class': 'artdeco-card ember-view relative break-words pb3 mt2'})
    education = []
    honor = []
    langueage = []
    name = []
    cur_pos = []
    position = []
    about = []
    img = []
    publication = []
    experience = [experiences(soup_exp)]
    skill = [skills(soup_skill)]
    profile = [name, cur_pos, position, about, education,honor,publication, langueage, img, experience,skill ]
    name.append(name_)
    cur_pos.append(cur_pos_)
    position.append(position_)
    about.append(about_)
    img.append(img_)
    
    for i in range(len(info)):
        section = info[i].find_all('div', attrs={'class': 'pvs-header__title-container'})[0].find_all('span')[0].text
        if(section == 'Education'):
            education.append(educations(info[i]))
        elif(section == 'Honors & awards'):
            honor.append(honors(info[i]))
        elif (section == 'Languages'):
            langueages = info[i].find_all('span', attrs={'class': 'mr1 t-bold'})
            langueages = list(map(lambda x: x.find_all('span')[0].text, langueages))
            langueages = json.dumps(langueages)
            langueage.append(langueages)
        elif (section == 'Publications'):
            publication.append(publications(info[i]))
    for section in profile:
        if len(section) == 0:
            section.append([])
    df = pd.DataFrame({'Name': name,'Job Title':cur_pos,'Location':position,'Image': img,'About': about  , 'Education':education, 'Experience': experience, 'Skill': skill,'Honors & awards':honor,'Publication': publication,'Languages':langueage })
    return df

In [49]:
def load_data(df):
    df_insert = ("""
    INSERT INTO profile(Name, `Job Title`,`Location` ,`Image` ,`About` ,`Education` ,`Experience` ,`Skill`,`Honors & awards`,`Publication`, `Languages`)
    VALUES ( 
    """)
    for col in df.columns:
        df[col] = df[col].astype('string')
        df_insert +=  "'"+ df.iloc[0][col] +"'"  + ','
    t = df_insert[:-2] +"')"
    conn.execute(t)

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(ChromeDriverManager().install())
login(browser)
count = 0
for i in  range(len(users.Profile)):
        print('Scraping profile of user: ', users.Name[0] ,' index ', i )
        soup, soup_exp, soup_skill = user_scrape_profile(browser, users.Profile[i]) 
        df = extract_profile_sections(soup, soup_exp, soup_skill)
        load_data(df)
        print('Sucessfully uploading profile of user: ', users.Name[0] ,' index ', i )
        count = count +1